# PySpark

In [ ]:
import findspark
findspark.init() 

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('CausalNex').getOrCreate()

path='../datasets/student-por.csv'

data=spark.read.csv(path, header=True, inferSchema=True, sep=';')

data.show(5)

In [ ]:
drop_cols=['school', 'sex', 'age', 'Mjob', 'Fjob', 'reason', 'guardian']

data=data.select([c for c in data.columns if c not in drop_cols])

data

In [ ]:
non_numeric_columns=[item[0]for item in data.dtypes if item[1].startswith('string')]
non_numeric_columns

In [ ]:
struct_data=data.select('*')
struct_data

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType


indexers=StringIndexer(inputCols=non_numeric_columns, 
                       outputCols=[c+'_' for c in non_numeric_columns],
                       stringOrderType='alphabetAsc')

struct_data=indexers.fit(struct_data).transform(struct_data)

struct_data=struct_data.select([c for c in struct_data.columns if c not in non_numeric_columns])

for c in struct_data.columns:
    struct_data=struct_data.withColumn(c, struct_data[c].cast(IntegerType()))

    
struct_data.toPandas()

# Multiprocessing

In [ ]:
def square(x):
    return x**2

In [ ]:
data=[i for i in range(1000000)]

In [ ]:
%%time
seq=[square(x) for x in data]
seq[:10]

In [ ]:
import multiprocessing as mp
import numpy as np

In [ ]:
mp.cpu_count()

In [ ]:
%%time
pool=mp.Pool(mp.cpu_count())   # grupo con todos los nucleos

res=pool.map(square, data)
pool.close()
res[:10]

In [ ]:
res=[]
for e in data:
    res.append(e**2)
res[:10]

In [ ]:
list(map(square, data))[:10]

In [ ]:
array=np.random.randint(0, 10, size=[200, 5000])
data=array.tolist()

In [ ]:
def rango(r, mini, maxi):
    c=0
    for e in r:
        if mini<=e<=maxi:
            c=+1
    return c

In [ ]:
%time res=[rango(e, 4, 8) for e in data]
res[:10]

In [ ]:
pool=mp.Pool()

%time res=pool.starmap(rango, [rango(e, 4, 8) for e in data])
pool.close()

In [ ]:
def rango2(i, r, mini, maxi):
    c=0
    
    for e in r:
        if mini<=e<=maxi:
            c=+1
            
    return i, c

In [ ]:
res=[]

def colecta(resul):
    global res
    res.append(resul)

In [ ]:
pool=mp.Pool(3)

for i,e in enumerate(data):
    pool.apply_async(rango2, args=(i,e,4,8), callback=colecta)
    
pool.close()
pool.join()

res.sort(key=lambda x: x[0])

res[:10]

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(np.random.randint(3, 10, size=[5, 2]))
df

In [ ]:
def hipo(c):
    return (c[1]**2+c[2]**2)**0.5

In [ ]:
with mp.Pool(3) as pool:
    res=pool.imap(hipo, df.itertuples(name=None), chunksize=10)
    out=[round(e, 2) for e in res]
out

# Pathos

In [ ]:
from pathos.multiprocessing import ProcessingPool as Pool

In [ ]:
df

In [ ]:
def fun(df):
    return df.shape

In [ ]:
cores=mp.cpu_count()

df_split=np.array_split(df, cores, axis=0)

pool=Pool(cores)

df_out=pd.DataFrame(np.vstack(pool.map(fun, df_split)))

pool.close()
pool.join()
pool.clear()

In [ ]:
df_out

# Dask

In [ ]:
# !pip3 install "dask[complete]"
# !pip3 install dask_ml

In [ ]:
import os
import dask
import dask.dataframe as dd

In [ ]:
archivos=os.path.join('../datasets', 'accounts.*.csv')
print(archivos)

In [ ]:
df=dd.read_csv(archivos)
df.head()

In [ ]:
df=dd.read_csv(os.path.join('../datasets', 'nycflights', '*.csv'),
               parse_dates={'Date':[0,1,2]})
df.head()

In [ ]:
df.tail()

In [ ]:
df=dd.read_csv(os.path.join('../datasets', 'nycflights', '*.csv'),
               parse_dates={'Date':[0,1,2]},
               dtype={'TailNum': str, 'CRSElapsedTime':float, 'Cancelled':bool})


df.tail()

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.shape

In [ ]:
df.DepDelay.max().compute()

In [ ]:
df.DepDelay.max().visualize(filename='../images/max_dask.png')

In [ ]:
len(df)

In [ ]:
len(df[~df.Cancelled])    # virgulilla ==> lo contrario (como el no logico)

In [ ]:
df.groupby('Origin').DepDelay.mean().compute()

In [ ]:
no_cancelado=df[~df.Cancelled]

media_delay=no_cancelado.DepDelay.mean()
std_delay=no_cancelado.DepDelay.std()

In [ ]:
std_delay

In [ ]:
%time media_delay.compute()

In [ ]:
%time std_delay.compute()

In [ ]:
%time media_delay_res, std_delay_res = dask.compute(media_delay, std_delay)

In [ ]:
media_delay_res

In [ ]:
std_delay_res

In [ ]:
type(std_delay_res)

In [ ]:
dask.visualize(media_delay, std_delay, filename='../images/std_dask.png')

# HDFS y Parquet  (sistemas de archivos)

In [ ]:
%time df_csv=dd.read_csv(archivos)
df_csv.head()

In [ ]:
target=os.path.join('../datasets', 'accounts.h5')

In [ ]:
%time df_csv.to_hdf(target, key='../datasets')

In [ ]:
%time df_h=dd.read_hdf(target, key='../datasets')
df_h.head()

In [ ]:
df_h.info(memory_usage='deep')

In [ ]:
%time df_h.amount.sum().compute()

In [ ]:
# parquet
# !pip3 install fastparquet

In [ ]:
target=os.path.join('../datasets', 'accounts.parquet')

df_csv.categorize(columns=['names']).to_parquet(target,
                                                storage_options={'has_nulls':True},
                                                engine='fastparquet')

In [ ]:
%time df_p=dd.read_parquet(target)
df_p.head()

In [ ]:
df_csv.info(memory_usage='deep')

In [ ]:
df_p.info(memory_usage='deep')

In [ ]:
%time df_csv.amount.sum().compute()

In [ ]:
%time df_p.amount.sum().compute()

# tqdm

In [ ]:
# !pip3 install tqdm

In [ ]:
from tqdm import trange

In [ ]:
import time
from tqdm.notebook import tqdm

In [ ]:
for i in tqdm(range(2), desc='Primer loop'):
    tqdm._instances.clear()
    for j in tqdm(range(5), desc='Segundo loop'):
        time.sleep(.5)

In [ ]:
for i in trange(2):
    tqdm._instances.clear()
    for j in trange(5):
        time.sleep(.5)

In [ ]:
df=pd.DataFrame(np.random.randint(0, int(1e8), (10000, 1000)))
df.head()

In [ ]:
tqdm.pandas()

df.groupby(0).progress_apply(lambda x: x**2)

In [ ]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [ ]:
res=[]

def fun(x):
    time.sleep(x)
    return x

for i in trange(10):
    res.append(delayed(fun)(i))
    
with ProgressBar():
    compute(res)

In [ ]:
res[5].compute()

# Vaex

https://towardsdatascience.com/how-to-analyse-100s-of-gbs-of-data-on-your-laptop-with-python-f83363dda94

**Data:** https://vaex.s3.us-east-2.amazonaws.com/taxi/yellow_taxi_2009_2015_f32.hdf5

In [ ]:
import vaex

import numpy as np
import pylab as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# tamaño del archivo
!ls -l -h ../../data/yellow_taxi_2009_2015_f32.hdf5

In [ ]:
%%time
# lectura archivo
df = vaex.open('../../data/yellow_taxi_2009_2015_f32.hdf5')

In [ ]:
%%time
df.describe()

In [ ]:
%%time
df.plot_widget(df.pickup_longitude, 
               df.pickup_latitude, 
               shape=512, 
               limits='minmax',
               f='log1p', 
               colormap='plasma')